<a href="https://colab.research.google.com/github/20195114/Detectron/blob/main/2024_03_25_%EC%8B%A4%EC%8A%B5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas
import pandas as pd

In [3]:
data = pd.read_csv("/content/kakao.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       5391 non-null   object 
 1   Open       5385 non-null   float64
 2   High       5385 non-null   float64
 3   Low        5385 non-null   float64
 4   Close      5385 non-null   float64
 5   Adj Close  5385 non-null   float64
 6   Volume     5385 non-null   float64
dtypes: float64(6), object(1)
memory usage: 294.9+ KB


In [5]:
high_price = data['High'].values
low_price = data['Low'].values

mid_price = (high_price + low_price)/2

In [7]:
# timestep을 50으로 설정해서 데이터를 생성
data_divideed = 50
day_length = data_divideed + 1
day_result = []

for i in range(len(mid_price) - day_length):
  day_result.append(mid_price[i: i + day_length])

print("전체 데이터 length:" , len(data))
print("나눈 데이터 length:" , len(day_result))
print(day_result[0])
print(day_result[1])

전체 데이터 length: 5391
나눈 데이터 length: 5340
[54600. 53850. 47650. 44350. 50250. 48150. 42950. 41300. 36550. 39150.
 41950. 38850. 37350. 36800. 36800. 32300. 30950. 29400. 31400. 33400.
 36500. 33600. 29600. 28800. 29700. 27450. 23900. 21800. 23600. 22900.
 22100. 22450. 23350. 21600. 20600. 22400. 19920. 17910. 17560. 18380.
 19190. 21750. 21650. 19400. 19400. 17740. 15040. 14910. 14100. 12830.
 13170.]
[53850. 47650. 44350. 50250. 48150. 42950. 41300. 36550. 39150. 41950.
 38850. 37350. 36800. 36800. 32300. 30950. 29400. 31400. 33400. 36500.
 33600. 29600. 28800. 29700. 27450. 23900. 21800. 23600. 22900. 22100.
 22450. 23350. 21600. 20600. 22400. 19920. 17910. 17560. 18380. 19190.
 21750. 21650. 19400. 19400. 17740. 15040. 14910. 14100. 12830. 13170.
 13380.]


In [14]:
# 훈련 데이터 와 테스트 데이트 분할

# 훈련 데이터 비율 설정
train_data_rate = 0.9

# 경계값 인덱스 구하기
boundary = round(day_result.shape[0] * train_data_rate)

train_data = day_result[:boundary, :]
test_data = day_result[boundary:, :]

print()

In [10]:
import numpy as np

In [11]:
day_result = np.array(day_result)

In [12]:
# 훈련 데이터 와 테스트 데이트 분할

# 훈련 데이터 비율 설정
train_data_rate = 0.9

# 경계값 인덱스 구하기
boundary = round(day_result.shape[0] * train_data_rate)

train_data = day_result[:boundary, :]
test_data = day_result[boundary:, :]

print()

In [15]:
# 경계값 인덱스를 기준으로 훈련 데이터 와 검증 데이터 분할

train_data = day_result[:boundary, :]
test_data = day_result[boundary:, :]

X_train = train_data[:, :-1]
#
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)

y_train = train_data[:, -1]
print(y_train.shape)

X_test = test_data[:, :-1]
X_test = np.reshape(X_test, (X_train.shape[0], X_train.shape[1], 1))
y_test = test_data[:, -1]


(4806, 50, 1)
(4806,)


ValueError: cannot reshape array of size 26700 into shape (4806,50,1)

In [16]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (4806, 50, 1)
X_test shape: (534, 50)


In [17]:
if X_train.shape[1] != X_test.shape[1]:
    raise ValueError("Inconsistent dimensions between X_train and X_test")

In [18]:
X_test = np.reshape(X_test, (X_train.shape[0], X_train.shape[1], 1))

ValueError: cannot reshape array of size 26700 into shape (4806,50,1)

In [19]:
# NLP
# 공통 코드
# 차원 축소
# 공통 코드
import sys
# sklearn ≥0.20 필수
import sklearn
# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os
# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)
# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import platform
from matplotlib import font_manager, rc
#매킨토시의 경우
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
#윈도우의 경우
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "dim_reduction"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
import warnings
warnings.filterwarnings(action='ignore')


# 차원 축소
# 공통 코드
import sys
# sklearn ≥0.20 필수
import sklearn
# 공통 모듈 임
from sklearn.datasets import make_blobs



# 텍스트 전처리
# 문장 토큰화
import nltk
nltk.download('punkt')

from nltk import sent_tokenize
test_sample = '안녕하세요. 반갑습니다. 어서오세요! 환영합니다.'
#. 나 !, ? 처럼 문장의 끝은 나타내는 기호나 \n 단위로 분할해서 list 로 리턴
sentences = sent_tokenize(text = test_sample)
print(sentences)
print(type(sentences))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['안녕하세요.', '반갑습니다.', '어서오세요!', '환영합니다.']
<class 'list'>


In [21]:
# 토큰화 및 문장 인코딩
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [
    '영실이는 나를 정말 정말 좋아해',
    '영실이는 영화를 무척 좋아해'
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

# Access the word_index dictionary from the tokenizer object
word_index = tokenizer.word_index

# Print the tokenizer and word_index
print(tokenizer)
print(word_index)

{'영실이는': 1, '정말': 2, '좋아해': 3, '나를': 4, '영화를': 5, '무척': 6}


In [22]:
# 문장 인코딩
word_encoding = tokenizer.texts_to_sequences(sentences)
print(word_encoding)

[[1, 4, 2, 2, 3], [1, 5, 6, 3]]


In [ ]:
# OOV - 사전에 없는 단어
new_sequences = ['영실이는 아담을 좋아해']
# 사전에 없는 단어는 기본적으로 무시됨
word_encoding = tokenizer.texts_to_sequences(new_sentences)
print(word_encoding)

In [25]:
print('new_sentences' in globals())

False


In [26]:
new_sentences = ['영실이는 아담을 좋아해']

In [27]:
word_encoding = tokenizer.texts_to_sequences(new_sentences)
print(word_encoding)

[[1, 3]]


In [28]:
tokenizer = Tokenizer(oov_token='<oov>')
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)

word_encoding = tokenizer.texts_to_sequences(new_sentences)
print(word_encoding)

{'<oov>': 1, '영실이는': 2, '정말': 3, '좋아해': 4, '나를': 5, '영화를': 6, '무척': 7}
[[2, 1, 4]]


In [30]:
tokenizer = Tokenizer(oov_token='<oov>', num_words = 3)
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)
# 문장을 인코딩 할 때 3개의 단어만을 사용하고 나머지는 모두 oov
word_encoding = tokenizer.texts_to_sequences(new_sentences)
print(word_encoding)

{'<oov>': 1, '영실이는': 2, '정말': 3, '좋아해': 4, '나를': 5, '영화를': 6, '무척': 7}
[[2, 1, 1]]


In [32]:
# 패딩 - 길이 맞추기
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = [
    '영실이는 아담을 정말 정말 좋아해',
    '영실이는 영화를 좋아해'
]

word_
print(pad_sequences(word_encoding))

print(pad_sequences(word_encoding, padding = 'post'))


[[2 1 1]]
[[2 1 1]]


In [ ]:
# 데이터 읽어오기
train = pd.read_csv('', sep = "\t")
train.head()

In [34]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 43.3 MB/s eta 0:00:00


In [36]:
import konlpy
from konlpy.tag import Kkma, Komoran, Okt

# Create an instance of the Okt tagger
okt = Okt()

In [38]:
text= "시리야오늘날씨어때?"
# 형태소 분석을 수행해서 언어와 품사를 리턴
print(Kkma.pos(text))
print(komoran.pos(text))
print(okt.pos(text))


TypeError: Kkma.pos() missing 1 required positional argument: 'phrase'

In [39]:
!pip install konlpy
!pip install kss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kss: filename=kss-4.5.4-py3-none-any.whl size=54464 sha256=93ff20964538b52fc78c414005ba2112371e6da4dffe475321b87aead7081472
  Stored in directory: /root/.cache/pip/wheels/61/7b/ba/e620ef5d96a61cdd83bdee4c2bb4aec8a74de5d72fcbb00e80
  Created wheel for pecab: filename=pecab-1.0.8-py3-none-any.whl size=26646665 sha256=dad7fe0f59ad4196114ae9c985f92336c4fd0b5e655cf6ef210509b1aef348bd
  Stored in directory: /root/.cache/pip/wheels/5c/6f/b4/ab61b8863d7d8b1409def8ae31adcaa089fa91b8d022ec309d
Successfully built kss pecab


In [40]:
from konlpy.tag import Kkma, Komoran, Okt

In [41]:
text = "시리야오늘날씨어때?"
# 형태소 분석을 수행해서 언어와 품사를 리턴
print(Kkma().pos(text))
print(Komoran().pos(text))
print(Okt().pos(text))

[('시', 'NNG'), ('리', 'NNG'), ('야', 'JX'), ('오늘날', 'NNG'), ('씨', 'NNB'), ('어', 'VV'), ('어', 'ECS'), ('때', 'NNG'), ('?', 'SF')]
[('시', 'NNB'), ('리야', 'NNP'), ('오늘날', 'NNP'), ('씨', 'NNB'), ('어떻', 'VA'), ('어', 'EF'), ('?', 'SF')]
[('시리', 'Noun'), ('야', 'Josa'), ('오늘날', 'Noun'), ('씨', 'Suffix'), ('어때', 'Adjective'), ('?', 'Punctuation')]


In [46]:
# 영어 와 한글만 남겨두고 삭제
# 자연어를 이용해서 감성 분석 같은 작업을 할 떄 대부분의 경ㅇ우는 숫자를 제거
train['document'] = train['document'].str.replace("[^A-Za-z가-힣ㄱ-ㅎ ㅏ - ㅣ]", "")
trian['document'].head()

NameError: name 'train' is not defined

In [47]:
train = train.dropna()
train.shape

NameError: name 'train' is not defined

In [44]:
from konlpy.tag import Okt
okt = Okt()

def word_tokenization(text):
  stop_words = ['는', '을', '를', '이', '와']
  return [word for word in okt.morphs(text) if word not in stop_words]
  # 형태소 분석 과 불용어 제거를 수행해서 문장을 단어의 list 로 변환
data = train['document'].apply(lambda x : word_tokenization(x))
data.head()




NameError: name 'train' is not defined

In [ ]:
training_size = 120000

train_labels = train["label"][:training_size]
valid_labels = train["label"][training_size:]

In [ ]:
# 토큰화 - 단어 사전을 만들고 문장을 숫자로 변환
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)

print("단어 개수:", len(tokenizer.word_index))

In [ ]:
# 5회 이상 등장하지 않은 단어 제거
def get_vocab_size(threshold):
  cnt = 0
  for x in tokenizer.word_counts.values():
    if x > threshold:
      cnt = cnt+1
  return cnt

print("5회 이상 등장한 단어의 개수:", get_vocab_size(5))

In [ ]:
oov_tok = "<oov>"
vocab_size = 15000

tokenizer = Tokenizer(oov_token = oov_tok, num_words = vocab_size)
tokenizer.fit_on_texts(data)
print("단어 개수:", len(tokenizer.word_counts))

In [ ]:
# 문장을 숫자로 변경
train_sequences = tokenizer.texts_to_sequences(train_sentences)
valid_sequences = tokenizer.texts_to_sequences(valid_sentences)

print(train_sentences[:3])
print(train_sequences[:3])

In [ ]:
# 문장 길이를 전부 동일하게 만들기
# 가장 긴 문장 길이를 확인
max_length = max(len(x) for x in train_sequences)
print(max_length)

In [ ]:
# 문장 길이가 최대 길이보다 짧을 경우 0을 채울 위치와 최대 길이보다 긴 경우
# 자를 위치를 설정
trunc_type = 'post'
padding_type = "post"

train_padded = pad_sequences(truncating = trunc_type, padding = padding_type, maxien = max_length)
valid_padded = pad_sequences(truncating = trunc_type, padding = padding_type, maxien = max_length)

train_labels = np.asarray(train_labels)
valid_labels = np.asarray(valid_labels)

In [ ]:
# 원래는 아 더빙... 진짜 짜증나네요 목소리 텍스트 였는데 형태소 분석을 해서
# 단어의 list 로 변경한 후 숫자로 변경하고 일정한 길이로 맞추는 작업을 수행
print(train_sentences[:1])
print(train_sequences[:1])
print(train_padded[:1])
